In [ ]:
# Fix randomness and hide warnings
seed = 42
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'
os.environ['TF_GPU_THREAD_MODE']="gpu_private"
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)
import numpy as np
np.random.seed(seed)
from scipy.interpolate import interp1d
import logging
import gc
import random
random.seed(seed)
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
plt.rc('font', size=16)
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as pyplot
# Import tensorflow
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
from tensorflow.keras import backend as K
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)
from tensorflow.keras import mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)
#tf.keras.backend.set_floatx('float16')
tf.function(jit_compile=True)
tf.keras.backend.floatx()

# Data Loading - Preprocessing

In [ ]:
def build_sequences_filtered(target_data, valid_periods, window=200, stride=50, telescope=18):
    assert window % stride == 0
    dataset = []
    labels = []
    for i, signal in enumerate(target_data):
        if valid_periods[i][1]-valid_periods[i][0] >= telescope*2:
            for j in np.arange(min(valid_periods[i][0],len(signal)-window-telescope),len(signal)-window-telescope,stride):
                input_sequence = signal[j:j+window]
                output_sequence = signal[j+window:j+window+telescope]
                dataset.append(input_sequence)
                labels.append(output_sequence)
    dataset = np.array(dataset)
    labels = np.array(labels)
    medians = np.median(dataset,axis=1)
    median_labels_telescope = np.median(labels,axis=1)
    return dataset, labels, medians, median_labels_telescope#, iqr_X, iqr_y

In [ ]:
telescope = 18
batch_size = 128
epochs = 200
window = 200
stride = 10
data = np.load("/kaggle/input/time-series-anndl/training_data.npy")
categories = np.load("/kaggle/input/time-series-anndl/categories.npy")
valid_periods = np.load("/kaggle/input/time-series-anndl/valid_periods.npy")
X, y, median_X, median_y = build_sequences_filtered(data, valid_periods, window, stride, telescope)
norm_X, norm_y = (X-median_X[:,None]), (y-median_y[:,None])#/iqr_y[:,None]
print(X.shape, y.shape)

In [ ]:
X_train, X_val, y_train, y_val, norm_X_train, norm_X_val, norm_y_train, norm_y_val, median_y_train, median_y_val = train_test_split(X, y, norm_X, norm_y, median_y, test_size=0.2, random_state=42)

In [ ]:
def make_dataset(X, y, batch_size=128, prefetch_amt=tf.data.experimental.AUTOTUNE):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    # reshape x from (200) to (200,1) and y from (9) to (9,1)
    dataset = dataset.map(lambda x, y: (tf.reshape(x, (200,1)), y),num_parallel_calls=tf.data.AUTOTUNE)
    #dataset = dataset.shuffle(buffer_size=1024).batch(batch_size, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)
    dataset = dataset.batch(batch_size, drop_remainder=False)
    dataset = dataset.cache()
    dataset = dataset.prefetch(prefetch_amt)
    return dataset

# T2V + GRU Model

In [ ]:
class T2V(tfkl.Layer):
    def __init__(self, output_dim=None, **kwargs):
        self.output_dim = output_dim
        super(T2V, self).__init__(**kwargs)
        
    def build(self, input_shape):
        self.W = self.add_weight(name='W',shape=(input_shape[-1], self.output_dim),initializer='uniform',trainable=True)
        self.P = self.add_weight(name='P',shape=(input_shape[1], self.output_dim),initializer='uniform',trainable=True)
        self.w = self.add_weight(name='w',shape=(input_shape[1], 1),initializer='uniform',trainable=True)
        self.p = self.add_weight(name='p',shape=(input_shape[1], 1),initializer='uniform',trainable=True)
        super(T2V, self).build(input_shape)
        
    def call(self, x):
        original = self.w * x + self.p
        sin_trans = K.sin(K.dot(x, self.W) + self.P)
        return K.concatenate([sin_trans, original], -1)

In [ ]:
multi_lstm_model = tfk.Sequential([ #28,312 after 10 epochs
    # Shape [batch, time, features] => [batch, lstm_units].
    # Adding more `lstm_units` just overfits more quickly.
    T2V(16),
    tfkl.GRU(150, dropout=0.2, return_sequences=True),
    tfkl.GRU(150, dropout=0.3, return_sequences=False),
    tfkl.Flatten(),
    tfkl.Dense(telescope,kernel_initializer=tf.initializers.zeros(), activation="gelu"),
])
multi_lstm_model.compile(loss=tfk.losses.MeanSquaredError(), metrics=['mae'], optimizer=tfk.optimizers.AdamW())
multi_lstm_model.build((None,window,1))
multi_lstm_model.summary() 
history = multi_lstm_model.fit(
    make_dataset(X_train, y_train, batch_size=batch_size), #normalized_X_train
    epochs = epochs,
    validation_data=make_dataset(X_val, y_val, batch_size=batch_size),#(X_val,y_val),# (normalized_X_val,normalized_y_val)
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=12, min_delta=5e-6, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=4, factor=0.1, min_lr=1e-7)
    ]
).history

In [ ]:
# Plot the training
plt.figure(figsize=(15,5))
plt.plot(history['loss'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(history['val_loss'], label='Custom ResNet', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('MSE')
plt.grid(alpha=.3)

plt.show()

In [ ]:
multi_lstm_model.evaluate(X_val, y_val)

In [ ]:
multi_lstm_model.save("/kaggle/working/T2V_Custom")

In [ ]:
!zip -r Model.zip /kaggle/working/T2V_Custom

In [ ]:
from IPython.display import FileLink
FileLink(r'Model.zip')